<a href="https://colab.research.google.com/github/himanshudas13/MovieRecommenderSystem/blob/main/Collaborative_Filtering_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install recommender-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 3.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import tensorflow as tf

In [ ]:
path='/content/drive/MyDrive/ML-DL/movies/ml-latest-small/ml-latest-small/ratings.csv'

In [30]:
def load_movie_list(path):
  user_ratings= pd.read_csv(path)
  user_ratings = user_ratings.sample(n=100, axis=0)
  user_ratings.columns = user_ratings.columns.str.replace("'", "")
  user_ratings.drop('timestamp', axis=1, inplace=True)
  user_ratings_pivot = user_ratings.pivot(index='movieId', columns='userId', values='rating').fillna('NA')
  user_ratings_pivot = user_ratings_pivot.sample(n=50, axis=1)
  return user_ratings_pivot

df=load_movie_list(path)
df.shape

(9724, 610)

In [ ]:
# movieList, movieList_df = load_Movie_List_pd()

# my_ratings = np.zeros(num_movies)          #  Initialize my ratings

# # Check the file small_movie_list.csv for id of each movie in our dataset
# # For example, Toy Story 3 (2010) has ID 2700, so to rate it "5", you can set
# my_ratings[2700] = 5

# #Or suppose you did not enjoy Persuasion (2007), you can set
# my_ratings[2609] = 2;

# # We have selected a few movies we liked / did not like and the ratings we
# # gave are as follows:
# my_ratings[929]  = 5   # Lord of the Rings: The Return of the King, The
# my_ratings[246]  = 5   # Shrek (2001)
# my_ratings[2716] = 3   # Inception
# my_ratings[1150] = 5   # Incredibles, The (2004)
# my_ratings[382]  = 2   # Amelie (Fabuleux destin d'Amélie Poulain, Le)
# my_ratings[366]  = 5   # Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)
# my_ratings[622]  = 5   # Harry Potter and the Chamber of Secrets (2002)
# my_ratings[988]  = 3   # Eternal Sunshine of the Spotless Mind (2004)
# my_ratings[2925] = 1   # Louis Theroux: Law & Disorder (2008)
# my_ratings[2937] = 1   # Nothing to Declare (Rien à déclarer)
# my_ratings[793]  = 5   # Pirates of the Caribbean: The Curse of the Black Pearl (2003)
# my_rated = [i for i in range(len(my_ratings)) if my_ratings[i] > 0]

# print('\nNew user ratings:\n')
# for i in range(len(my_ratings)):
#     if my_ratings[i] > 0 :
#         print(f'Rated {my_ratings[i]} for  {movieList_df.loc[i,"title"]}');

In [32]:
# Get all movie IDs from the DataFrame
import random
movie_ids = df.index.tolist()

# Create a dictionary for the new user's ratings
my_ratings = {}

# Generate 10 random movie IDs and assign random ratings (1-5)
for _ in range(100):
    movie_id = random.choice(movie_ids)
    rating = random.randint(1, 5)
    my_ratings[movie_id] = rating

# Add the new user's ratings to the DataFrame
new_user_id = 700  # As specified in the prompt
for movie_id, rating in my_ratings.items():
    df.loc[movie_id, new_user_id] = rating

print(f"New user_id: {new_user_id}\n{df}")

New user_id: 700
userId   1   2   3   4    5    6    7    8   9   10   ...  602  603  604  605  \
movieId                                               ...                       
1        4.0  NA  NA  NA  4.0   NA  4.5   NA  NA  NA  ...   NA  4.0  3.0  4.0   
2         NA  NA  NA  NA   NA  4.0   NA  4.0  NA  NA  ...  4.0   NA  5.0  3.5   
3        4.0  NA  NA  NA   NA  5.0   NA   NA  NA  NA  ...   NA   NA   NA   NA   
4         NA  NA  NA  NA   NA  3.0   NA   NA  NA  NA  ...   NA   NA   NA   NA   
5         NA  NA  NA  NA   NA  5.0   NA   NA  NA  NA  ...   NA   NA  3.0   NA   
...      ...  ..  ..  ..  ...  ...  ...  ...  ..  ..  ...  ...  ...  ...  ...   
193581    NA  NA  NA  NA   NA   NA   NA   NA  NA  NA  ...   NA   NA   NA   NA   
193583    NA  NA  NA  NA   NA   NA   NA   NA  NA  NA  ...   NA   NA   NA   NA   
193585    NA  NA  NA  NA   NA   NA   NA   NA  NA  NA  ...   NA   NA   NA   NA   
193587    NA  NA  NA  NA   NA   NA   NA   NA  NA  NA  ...   NA   NA   NA   NA   
193609    N

In [33]:
def normalize_ratings(df):
  df_ = df.replace('NA', np.nan)
  y_mean = df_.mean(axis=1, skipna=True)
  df_ = df_.subtract(y_mean, axis=1)
  return df_, y_mean

In [34]:
df_norm, y_mean = normalize_ratings(df)

In [35]:
df_norm

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.07907,NaN,NaN,NaN,0.928571,NaN,1.314815,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,0.053922,NaN,1.125,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.07907,NaN,NaN,NaN,NaN,1.053922,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,-0.946078,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,1.053922,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
num_movies, num_users = df_norm.shape
num_features = 10

# Set initial parameters (W, X), using random initialization
np.random.seed(1234)  # for consistent results
W = np.random.normal(size=(num_users, num_features))
X = np.random.normal(size=(num_movies, num_features))
b = np.random.normal(size=(1, num_users))

In [37]:


# Convert X, W, and b to TensorFlow variables
X = tf.Variable(X, dtype=tf.float64)
W = tf.Variable(W, dtype=tf.float64)
b = tf.Variable(b, dtype=tf.float64)

# Ensure df_norm is a TensorFlow tensor
df_norm = tf.convert_to_tensor(df_norm, dtype=tf.float64)


In [38]:
import tensorflow as tf

def cost_func(X, W, b, Y, lambda_):
    """
    Returns the cost for the content-based filtering using TensorFlow.

    Args:
      X (tf.Tensor (num_movies,num_features)): matrix of item features
      W (tf.Tensor (num_users,num_features)) : matrix of user parameters
      b (tf.Tensor (1, num_users))           : vector of user parameters
      Y (tf.Tensor (num_movies,num_users))   : matrix of user ratings of movies
      lambda_ (float): regularization parameter

    Returns:
      J (tf.Tensor) : Cost
    """
    # Create a mask for non-NA values in Y (i.e., where ratings exist)
    mask = ~tf.math.is_nan(Y)

    # Compute the prediction error
    prediction_error = (tf.matmul(X, tf.transpose(W)) + b - Y)

    # Apply the mask to only consider valid ratings (ignore NaN values)
    masked_error = tf.where(mask, prediction_error, tf.zeros_like(prediction_error))

    # Calculate the cost function
    J = 0.5 * tf.reduce_sum(masked_error**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))

    return J


In [39]:
# Evaluate cost function with regularization
J = cost_func(X, W, b, df_norm, 1.5);
print(f"Cost (with regularization): {J:0.2f}")

Cost (with regularization): 691041.90


In [1]:
import tensorflow as tf
from tensorflow import keras

# Set up your training variables
iterations = 200
batch_size = 10  # You can adjust this value based on your dataset size and memory
lambda_ = 1
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

# Assuming X, W, b, and df_norm are defined as tf.Variable or tf.Tensor
dataset = tf.data.Dataset.from_tensor_slices((X, df_norm)).batch(batch_size)

for iter in range(iterations):
    for batch_X, batch_df_norm in dataset:
        # Use TensorFlow’s GradientTape to record the operations used to compute the cost
        with tf.GradientTape() as tape:
            # Compute the cost (forward pass included in cost)
            cost_value = cost_func(batch_X, W, b, batch_df_norm, lambda_)

        # Use the gradient tape to automatically retrieve the gradients of the trainable variables with respect to the loss
        grads = tape.gradient(cost_value, [W, b])

        # Run one step of gradient descent by updating the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, [W, b]))

    # Log periodically.
    if iter % 20 == 0:
        print(f"Training loss at iteration {iter}: {cost_value.numpy():.1f}")


NameError: name 'X' is not defined

In [ ]:
p = np.matmul(X.numpy(), np.transpose(W.numpy())) + b.numpy()

y_mean = y_mean.to_numpy()
# Replace NaN values in y_mean with 0 before restoring the mean
y_mean_filled = np.nan_to_num(y_mean.reshape(-1, 1), nan=0.0)

# Restore the mean to get the predicted ratings
pm = p + y_mean_filled

# Predictions for the new user (52 is the column corresponding to the new user)
my_predictions = pm[:, new_user_id - 1]

# Sort predictions in descending order
ix = np.argsort(my_predictions)[::-1]


# my_predictions[ix]

array([ 4.7895269 ,  4.7895269 ,  4.7895269 ,  4.7895269 ,  4.7895269 ,
        4.7895269 ,  4.7895269 ,  4.7895269 ,  4.2895269 ,  4.2895269 ,
        4.2895269 ,  4.2895269 ,  3.7895269 ,  3.7895269 ,  3.7895269 ,
        3.7895269 ,  3.7895269 ,  3.7895269 ,  3.7895269 ,  3.7895269 ,
        3.7895269 ,  3.7895269 ,  3.7895269 ,  3.7895269 ,  3.2895269 ,
        3.2895269 ,  3.2895269 ,  3.2895269 ,  3.2895269 ,  3.2895269 ,
        3.2895269 ,  3.2895269 ,  2.7895269 ,  2.7895269 ,  2.7895269 ,
        2.7895269 ,  2.7895269 ,  2.7895269 ,  2.7895269 ,  2.7895269 ,
        2.7895269 ,  2.7895269 ,  2.7895269 ,  2.7895269 ,  2.2895269 ,
        2.2895269 ,  2.2895269 ,  2.2895269 ,  2.2895269 ,  2.28952689,
        2.0395269 ,  1.7895269 ,  1.7895269 ,  1.7895269 ,  1.7895269 ,
        1.2895269 ,  1.2895269 ,  0.7895269 ,  0.2895269 ,  0.2895269 ,
        0.2895269 , -0.2104731 , -0.2104731 , -0.2104731 , -0.2104731 ,
       -0.2104731 , -0.2104731 , -0.2104731 , -0.2104731 , -0.21

In [ ]:


# Get movie IDs that the new user has already rated
my_rated = my_ratings.keys()

# Display top 17 movie predictions that the user hasn't rated yet
print("Top movie predictions for the new user:\n")
for i in range(17):
    j = ix[i]
    if movie_ids[j] not in my_rated:  # Check if the movie wasn't rated by the new user
        print(f'Predicting rating {my_predictions[j]:0.2f} for movie {movie_ids[j]}')

# Compare original and predicted ratings for movies the user has rated
print('\n\nOriginal vs Predicted ratings:\n')
for movie_id in my_rated:
    idx = movie_ids.index(movie_id)
    original_rating = my_ratings[movie_id]
    predicted_rating = my_predictions[idx]
    print(f'Original {original_rating}, Predicted {predicted_rating:0.2f} for {movie_ids[idx]}')


Top movie predictions for the new user:

Predicting rating 4.79 for movie 785
Predicting rating 4.79 for movie 356
Predicting rating 4.79 for movie 194
Predicting rating 4.79 for movie 5952
Predicting rating 4.79 for movie 593
Predicting rating 4.79 for movie 1201
Predicting rating 4.79 for movie 1722
Predicting rating 4.79 for movie 3897
Predicting rating 4.29 for movie 293
Predicting rating 4.29 for movie 179819
Predicting rating 4.29 for movie 69951
Predicting rating 4.29 for movie 2028
Predicting rating 3.79 for movie 454
Predicting rating 3.79 for movie 97188
Predicting rating 3.79 for movie 1167
Predicting rating 3.79 for movie 4310


Original vs Predicted ratings:

Original 3, Predicted 3.29 for 56174
Original 3, Predicted 2.29 for 2702
Original 4, Predicted 3.79 for 27808
Original 4, Predicted 2.04 for 231
Original 1, Predicted 2.29 for 4308
Original 3, Predicted 2.79 for 95
Original 1, Predicted 0.79 for 2546
Original 3, Predicted 2.79 for 316
Original 2, Predicted 1.79 for 15

In [27]:
# Create a dictionary to store movie IDs and their corresponding number of ratings and predicted ratings
movie_info = {}

for i in range(len(my_predictions)):
    movie_id = movie_ids[i]
    predicted_rating = my_predictions[i]

    # Use .loc to access rows by label (movie_id)
    try:
        # Count how many values in the row for the movie_id are not NaN
        num_ratings = df.loc[movie_id].notna().sum()
    except KeyError:
        # Handle the case where movie_id is not found in the index
        num_ratings = 0

    # Store the predicted rating and the number of ratings in the dictionary
    movie_info[movie_id] = {'predicted_rating': predicted_rating, 'num_ratings': num_ratings}

# Sort the movies based on the number of ratings and predicted ratings in descending order
sorted_movies = sorted(movie_info.items(), key=lambda item: (item[1]['num_ratings'], item[1]['predicted_rating']), reverse=True)

# Select the top 5 movies
top_5_movies = sorted_movies[:5]

# Display the top 5 movies
print("Top 5 movie recommendations based on number of ratings and highest predicted rating:")
for movie_id, info in top_5_movies:
    print(f"Movie ID: {movie_id}, Predicted Rating: {info['predicted_rating']:.2f}, Number of Ratings: {info['num_ratings']}")


Top 5 movie recommendations based on number of ratings and highest predicted rating:
Movie ID: 27808, Predicted Rating: 3.79, Number of Ratings: 51
Movie ID: 56174, Predicted Rating: 3.29, Number of Ratings: 51
Movie ID: 95, Predicted Rating: 2.79, Number of Ratings: 51
Movie ID: 316, Predicted Rating: 2.79, Number of Ratings: 51
Movie ID: 2702, Predicted Rating: 2.29, Number of Ratings: 51
